In [2]:
import pandas as pd
import os
import gene_tools as gt


os.chdir("C:/Users/alber/OneDrive/Documents/GitHub/ProjetBioinfo")

print(os.getcwd())
os.listdir("data")

C:\Users\alber\OneDrive\Documents\GitHub\ProjetBioinfo


['expl_analysis.png', '00_BPStart']

In [3]:
from gene_tools import foldersLoad, cleanDic, compute_scores, geneScores, NA_filtering,evaluate_trait_scores,evaluate_OR
from Scoring.scomet import Mean, Max, Min, Median, pca

root_path = os.path.join('data', '00_BPStart/00_gene_prioritization')

folders = ['eQTL_GWAS_blood', 'Exome', 'GWAS','pQTL_GWAS']
raw_data = foldersLoad(root_path, folders)
merged = pd.read_csv(r"data\00_BPStart\merged_drug_all_db_with_hgnc_TTD.tsv", sep="\t")
dic_clean = gt.cleanDic(raw_data)


In [6]:
from Scoring.scomet import Mean, Max, Min, Median, pca, Product

ldl_clean = dic_clean["LDL"]

Max(ldl_clean)
Mean(ldl_clean)
Median(ldl_clean)
Min(ldl_clean)
Product(ldl_clean)

score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"]
drug_targets = set(merged[merged['trait'] == 'LDL'].loc[merged[merged['trait'] == 'LDL']["Sum"] >= 3,"EnsemblId"])   
                                                                                                             
for col in score_cols:
    top = ldl_clean.loc[ldl_clean[col] < 0.5]
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")
    
for col in score_cols:
    top = ldl_clean.sort_values(by=col, ascending=True).head(int(0.01 * len(ldl_clean)))
    top_ids = set(top["EnsemblId"])
    overlap = top_ids & drug_targets
    percent = 100 * len(overlap) / len(top_ids)
    print(f"{col}: {percent:.2f}% of top 1% are drug targets")


    
len(ldl_clean["GWAS_percentile"].dropna())

Prioscore_mean: 20.00% of top 1% are drug targets
Prioscore_max: 7.69% of top 1% are drug targets
Prioscore_min: 6.01% of top 1% are drug targets
Prioscore_median: 22.22% of top 1% are drug targets
Prioscore_product: 9.35% of top 1% are drug targets
Prioscore_mean: 4.41% of top 1% are drug targets
Prioscore_max: 2.94% of top 1% are drug targets
Prioscore_min: 6.37% of top 1% are drug targets
Prioscore_median: 4.90% of top 1% are drug targets
Prioscore_product: 5.39% of top 1% are drug targets


19197

In [7]:
ldl_clean

,EnsemblId,Trait,eQTL_percentile,Exome_percentile,GWAS_percentile,pQTL_percentile,Prioscore_max,Prioscore_mean,Prioscore_median,Prioscore_min,Prioscore_product
0,ENSG00000130204,LDL,0.007948,47.153867,0.138584,NaN,47.153867,15.766800,0.138584,0.007948,-0.985879
1,ENSG00000113161,LDL,0.071536,0.053149,0.138584,NaN,0.138584,0.087756,0.071536,0.053149,-2.516161
2,ENSG00000134243,LDL,0.039742,90.193994,0.138584,NaN,90.193994,30.124107,0.138584,0.039742,-0.233217
3,ENSG00000189045,LDL,0.015897,1.498804,0.138584,NaN,1.498804,0.551095,0.138584,0.015897,-1.904413
4,ENSG00000142252,LDL,0.262300,77.517938,0.878569,NaN,77.517938,26.219602,0.878569,0.262300,0.960928
...,...,...,...,...,...,...,...,...,...,...,...
20428,ENSG00000160202,LDL,NaN,NaN,99.801276,NaN,99.801276,99.801276,99.801276,99.801276,4.603181
20429,ENSG00000224132,LDL,NaN,NaN,99.827424,NaN,99.827424,99.827424,99.827424,99.827424,4.603443
20430,ENSG00000228120,LDL,NaN,NaN,99.921556,NaN,99.921556,99.921556,99.921556,99.921556,4.604385
20431,ENSG00000205106,LDL,NaN,NaN,99.994770,NaN,99.994770,99.994770,99.994770,99.994770,4.605118


In [ ]:


from gene_tools.analysis import evaluate_OR

evaluate_OR(merged,ldl_clean,sum_threshold=3,top_percent = 0.005,score_cols = ["Prioscore_mean", "Prioscore_max", "Prioscore_min", "Prioscore_median","Prioscore_product"])


Prioscore_mean: OR = 1.62, p = 2.8244e-01, drug targets in top 0.5% = 6
Prioscore_max: OR = 1.62, p = 2.8244e-01, drug targets in top 0.5% = 6
Prioscore_min: OR = 3.83, p = 1.0388e-04, drug targets in top 0.5% = 13
Prioscore_median: OR = 3.16, p = 1.4382e-03, drug targets in top 0.5% = 11
Prioscore_product: OR = 3.49, p = 4.0380e-04, drug targets in top 0.5% = 12
